# Day 11 - Monkey in the Middle

https://adventofcode.com/2022/day/11

In [17]:
from pathlib import Path
from typing import Literal
import numpy as np
import ipytest
import math

ipytest.autoconfig(addopts=["-v"])

INPUTS = Path("input.txt").read_text().strip().split("\n\n")

## Part 1

In [18]:
class Monkey:
    def __init__(
        self,
        items: list[int],
        op_type: Literal["+", "*", "^2"],
        op_val: int,
        test_val: int,
        true_target: int,
        false_target: int,
    ):
        self.items = items
        self.op_type = op_type
        self.op_val = op_val
        self.test_val = test_val
        self.true_target = true_target
        self.false_target = false_target
        self.num_inspections = 0

    def do_inspections(
        self,
        worry_divisor: int | None = None,
        lcm: int | None = None,
    ) -> list[tuple[int, int]]:
        if not self.items:
            return []
        items_array = np.array(self.items, dtype=np.int64)
        if self.op_type == "^2":
            items_array = np.power(items_array, 2, dtype=np.int64)
        elif self.op_type == "*":
            items_array *= self.op_val
        else:
            items_array += self.op_val

        if worry_divisor:
            items_array //= worry_divisor
        if lcm:
            items_array %= lcm

        mods = items_array % self.test_val
        items_to_throw = [
            (self.false_target if mod else self.true_target, int(item))
            for item, mod in zip(items_array, mods)
        ]
        self.num_inspections += len(self.items)
        self.items = []
        return items_to_throw

In [19]:
def parse_inputs(inputs: list[str]) -> list[Monkey]:
    monkeys = []
    for details in inputs:
        # ignore row 0
        # (monkey identifier; we'll just have a list of Monkey class instances)
        m = details.split("\n")[1:]
        # items split from first row
        items = list(map(int, m[0].split("items: ")[1].split(", ")))
        # operation and its value from second row
        op_type, val = m[1].split("old ", maxsplit=1)[1].split()
        if val == "old":
            # Special case, this operation is actually squaring the value
            op_val = -1
            op_type = "^2"
        else:
            op_val = int(val)
        # test value from third row
        test_val = int(m[2].split("by ")[1])
        true_target = int(m[3].split()[-1])
        false_target = int(m[4].split()[-1])
        monkeys.append(
            Monkey(
                items=items,
                op_type=op_type,  # type: ignore
                op_val=op_val,
                test_val=test_val,
                true_target=true_target,
                false_target=false_target,
            )
        )
    return monkeys


monkeys = parse_inputs(INPUTS)

In [20]:
from copy import deepcopy


def run_rounds(
    monkeys: list[Monkey],
    num_rounds: int = 20,
    worry_divisor: int | None = None,
) -> list[Monkey]:
    monkeys_copy = deepcopy(monkeys)
    if worry_divisor:
        kwargs = {"worry_divisor": worry_divisor}
    else:
        # lcm required instead
        lcm = math.lcm(*[m.test_val for m in monkeys_copy])
        kwargs = {"lcm": lcm}
    for i in range(num_rounds):
        for monkey in monkeys_copy:
            for target, item in monkey.do_inspections(**kwargs):
                monkeys_copy[target].items.append(item)
    return monkeys_copy

In [21]:
monkeys_after_part1 = run_rounds(monkeys, worry_divisor=3)

top_monkeys = sorted(x.num_inspections for x in monkeys_after_part1)[-2:]
result = top_monkeys[0] * top_monkeys[1]
print(f"{result=}")

result=66124


## Part 2

I'll admit I was stuck here for a while before I checked some discussion in Python Discord about it. I didn't need much spoiling, but I did learn about [least common multiples](https://en.wikipedia.org/wiki/Least_common_multiple), which is what we'll need in order to keep these numbers below a manageable maximum. Otherwise they overflow and can't be correctly calculated anymore, throwing off the calculations for number of inspections even as low as 1,000 rounds, let alone 10,000.

But with that simple adjustment to incorporate `math.lcm`, we needed no further changes and could get a positive result.

In [22]:
%%ipytest

import pytest


TEST_INPUTS = """Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1"""

def test_part1():
    inputs = TEST_INPUTS.split('\n\n')
    monkeys = parse_inputs(inputs)
    monkeys_after_test = run_rounds(monkeys, num_rounds=20, worry_divisor=3)
    inspections = [x.num_inspections for x in monkeys_after_test]
    top_monkeys = sorted(inspections)[-2:]
    result = top_monkeys[0] * top_monkeys[1]
    expected = 10605
    assert result == expected, f"{result=}, {expected=}"


@pytest.mark.parametrize("num_rounds, expected_inspections", [
    (1, [2, 3, 4, 6]),
    (20, [97, 8, 99, 103]),
    (1_000, [5192, 5204, 199, 4792]),
    (2_000, [10391, 10419, 9577, 392]),
    (3_000, [15638, 14358, 587, 15593]),
    (4_000, [20858, 19138, 780, 20797]),
    (5_000, [26075, 23921, 974, 26000]),
    (6_000, [31294, 28702, 1165, 31204]),
    (7_000, [36508, 33488, 1360, 36400]),
    (8_000, [41728, 38268, 1553, 41606]),
    (9_000, [46945, 43051, 1746, 46807]),
    (10_000, [52166, 47830, 1938, 52013]),
])
def test_part2(num_rounds: int, expected_inspections: list[int]):
    inputs = TEST_INPUTS.split('\n\n')
    monkeys = parse_inputs(inputs)
    monkeys_after_test = run_rounds(monkeys, num_rounds=num_rounds)
    inspections = [x.num_inspections for x in monkeys_after_test]
    assert sorted(expected_inspections) == sorted(inspections)


======================================= test session starts ========================================
platform darwin -- Python 3.10.7, pytest-7.2.0, pluggy-1.0.0 -- /Users/garice/Library/Caches/pypoetry/virtualenvs/griceturrble-advent-of-code-8jQN35Cx-py3.10/bin/python
cachedir: .pytest_cache
rootdir: /Users/garice/dev/gits/personal/advent-of-code/2022/day11
collecting ... collected 13 items

t_9d9c4dd4a7b14c7d9203dea719b1e4ae.py::test_part1 PASSED                                     [  7%]
t_9d9c4dd4a7b14c7d9203dea719b1e4ae.py::test_part2[1-expected_inspections0] PASSED            [ 15%]
t_9d9c4dd4a7b14c7d9203dea719b1e4ae.py::test_part2[20-expected_inspections1] PASSED           [ 23%]
t_9d9c4dd4a7b14c7d9203dea719b1e4ae.py::test_part2[1000-expected_inspections2] PASSED         [ 30%]
t_9d9c4dd4a7b14c7d9203dea719b1e4ae.py::test_part2[2000-expected_inspections3] PASSED         [ 38%]
t_9d9c4dd4a7b14c7d9203dea719b1e4ae.py::test_part2[3000-expected_inspections4] PASSED         [ 46%]
t_9d

In [23]:
monkeys_after_part2 = run_rounds(monkeys, num_rounds=10_000)

In [24]:
top_monkeys = sorted(x.num_inspections for x in monkeys_after_part2)[-2:]
result = top_monkeys[0] * top_monkeys[1]
print(f"{result=}")

result=19309892877
